<a href="https://colab.research.google.com/github/microsoft/FLAML/blob/main/notebook/autogen_agent_human_feedback.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Interactive LLM Agent with Human Feedback

FLAML offers an experimental feature of interactive LLM agents, which can be used to solve various tasks, including coding and math problem-solving.

In this notebook, we demonstrate how to use `AssistantAgent` and `UserProxyAgent` to solve a challenging math problem with human feedback. Here `AssistantAgent` is an LLM-based agent that can write Python code (in a Python coding block) for a user to execute for a given task. `UserProxyAgent` is an agent which serves as a proxy for a user to execute the code written by `AssistantAgent`. By setting `human_input_mode` properly, the `UserProxyAgent` can also prompt the user for feedback to `AssistantAgent`. For example, when `human_input_mode` is set to "ALWAYS", the `UserProxyAgent` will always prompt the user for feedback. When user feedback is provided, the `UserProxyAgent` will directly pass the feedback to `AssistantAgent` without doing any additional steps. When no user feedback is provided, the `UserProxyAgent` will execute the code written by `AssistantAgent` directly and returns the execution results (success or failure and corresponding outputs) to `AssistantAgent`.

## Requirements

FLAML requires `Python>=3.7`. To run this notebook example, please install flaml with the [autogen] option:
```bash
pip install flaml[autogen]
```

In [13]:
# %pip install flaml[autogen]

## Set your API Endpoint

The [`config_list_gpt4_gpt35`](https://microsoft.github.io/FLAML/docs/reference/autogen/oai/openai_utils#config_list_gpt4_gpt35) function tries to create a list of gpt-4 and gpt-3.5 configurations using Azure OpenAI endpoints and OpenAI endpoints. It assumes the api keys and api bases are stored in the corresponding environment variables or local txt files:

- OpenAI API key: os.environ["OPENAI_API_KEY"] or `openai_api_key_file="key_openai.txt"`.
- Azure OpenAI API key: os.environ["AZURE_OPENAI_API_KEY"] or `aoai_api_key_file="key_aoai.txt"`. Multiple keys can be stored, one per line.
- Azure OpenAI API base: os.environ["AZURE_OPENAI_API_BASE"] or `aoai_api_base_file="base_aoai.txt"`. Multiple bases can be stored, one per line.

It's OK to have only the OpenAI API key, or only the Azure OpenAI API key + base.


In [14]:
from flaml import oai

config_list = oai.config_list_gpt4_gpt35()

The config list looks like the following:
```python
config_list = [
    {
        'model': 'gpt-4',
        'api_key': '<your OpenAI API key here>',
    },  # only if OpenAI API key is found
    {
        'model': 'gpt-4',
        'api_key': '<your first Azure OpenAI API key here>',
        'api_base': '<your first Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-03-15-preview',
    },  # only if the at least one Azure OpenAI API key is found
    {
        'model': 'gpt-4',
        'api_key': '<your second Azure OpenAI API key here>',
        'api_base': '<your second Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-03-15-preview',
    },  # only if the second Azure OpenAI API key is found
    {
        'model': 'gpt-3.5-turbo',
        'api_key': '<your OpenAI API key here>',
    },  # only if OpenAI API key is found
    {
        'model': 'gpt-3.5-turbo',
        'api_key': '<your first Azure OpenAI API key here>',
        'api_base': '<your first Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-03-15-preview',
    },  # only if the at least one Azure OpenAI API key is found
    {
        'model': 'gpt-3.5-turbo',
        'api_key': '<your second Azure OpenAI API key here>',
        'api_base': '<your second Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-03-15-preview',
    },  # only if the second Azure OpenAI API key is found
]
```

You can directly override it if the above function returns an empty list, i.e., it doesn't find the keys in the specified locations.

## Construct Agents

We construct the assistant agent and the user proxy agent.

In [15]:
from flaml.autogen.agent.assistant_agent import AssistantAgent
from flaml.autogen.agent.user_proxy_agent import UserProxyAgent

# create an assistant which is essentially a AssistantAgent instance named "coding_agent"
assistant = AssistantAgent(name="coding_agent", request_timeout=600, seed=42, config_list=config_list)
# create a UserProxyAgent instance named "user"
user = UserProxyAgent(
    name="user",
    human_input_mode="ALWAYS",
    is_termination_msg=lambda x: x.rstrip().endswith("TERMINATE"),
)

# the purpose of the following line is to log the conversation history
oai.ChatCompletion.start_logging()


## Perform a task

We invoke the `receive()` method of the assistant agent to start the conversation. When you run the cell below, you will be prompted to provide feedback after receving a message from the assistant agent. If you don't provide any feedback (by pressing Enter directly), the user proxy agent will try to execute the code suggested by the assistant agent on behalf of you, or terminate if the assistant agent sends a "TERMINATE" signal in the end of the message.

In [16]:
math_problem_to_solve = """
     Find $a + b + c$, given that $x+y \\neq -1$ and 
     \\begin{align}
	ax + by + c & = x + 7,\\
	a + bx + cy & = 2x + 6y,\\
	ay + b + cx & = 4x + y.
	\\end{align}.
    """

# the assistant receives a message from the user, which contains the task description
assistant.receive(math_problem_to_solve, user)


**** coding_agent received message from user ****


     Find $a + b + c$, given that $x+y \neq -1$ and 
     \begin{align}
	ax + by + c & = x + 7,\
	a + bx + cy & = 2x + 6y,\
	ay + b + cx & = 4x + y.
	\end{align}.
    

**** user received message from coding_agent ****

To solve the system of linear equations, we can use the Cramer's Rule of solving systems of linear equations. Let's write this in code:

```python
import numpy as np

A = np.array([[1, 1, 1], [1, 1, 1], [1, 1, 1]])
B = np.array([[1, 7], [2, 6], [4, 1]])

det_A = np.linalg.det(A)

if det_A != 0:
    x_solution = np.linalg.solve(A, B[:, 0])
    y_solution = np.linalg.solve(A, B[:, 1])
    a, b, c = x_solution[0], x_solution[1], x_solution[2]
    result = a + b + c
    print(round(result, 2))
else:
    print("There is no unique solution.")
```
Execute this code to find the values of $a, b, c$ and calculate the sum $a+b+c$.

**** coding_agent received message from user ****

Are you assuming x=y=1?

**** user received mes

## Analyze the conversation

The human user provided feedback such as "Are you assuming x=y=1?", "x=y=0". At some steps, the human user didn't provide feedback, and the code was executed. The executed results and error messages are returned to the coding agent, and the coding agent was able to modify the code to rectify. In the end, the task is complete and a "TERMINATE" signal is sent from the coding agent. The user skipped feedback in the end and the conversation is finished.

After the conversation is finished, we can save the log of the conversation between the two agents. The log can be accessed from `oai.ChatCompletion.logged_history`.

In [17]:
print(oai.ChatCompletion.logged_history)

{'[{"content": "You are a helpful AI assistant.\\n    In the following cases, suggest python code (in a python coding block) for the user to execute.\\n    1. When you need to ask the user for some info, use the code to output the info you need, for example, browse or search the web, download/read a file.\\n    2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly. Solve the task step by step if you need to.\\n    If you want the user to save the code in a file before executing it, put # filename: <filename> inside the code block as the first line. Don\'t suggest bash/shell/sh block. Don\'t include multiple code blocks in one response. Use \'print\' function for the output when relevant. Check the execution result returned by the user.\\n    If the result indicates there is an error, fix the error and output the code again.\\n    Reply \\"TERMINATE\\" in the end when the task is done.\\n    ", "role": "system"},

In [18]:
import json

json.dump(oai.ChatCompletion.logged_history, open("conversations.json", "w"), indent=2)